## Environment setup

In [1]:
import glob, os, time
from datetime import datetime
import importlib

import numpy as np

import tensorflow as tf

2023-04-06 21:54:54.824030: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Networks definitions and adaptations

In [2]:
from architecture_definitions import *

oneOverPt = False 
lut_nn = True
output_type = 0
last_input_is_bias = True

if output_type == 1:
    layer3_neurons = 3
    loss_fn = custom_loss3
else: 
    output_cnt = 1
    layer3_neurons = 1
    loss_fn = 'mae'
        
if not last_input_is_bias:
    networkInputSize =  nLayers
    layer2_lutRangesCnt = 1
    layer2_input_offset = None 
 
dir_postfix = get_lut_nn_dir_postfix() 
    
print_LUT_NN()  

LUT NN definitions:
layer1_lut_size 1024
layer2_lut_size 256
layer3_lut_size 32
layer2_lutRangesCnt 16
layer2_input_offset 8.0
------------------------


### Training data set preparation

In [3]:
import io_functions as io
importlib.reload(io)

batchSize = 4096
nEpochs = 1

#trainDataDir = "/scratch_ssd/akalinow/ProgrammingProjects/MachineLearning/OMTF/data/18_12_2020/"   
trainDataDir = "/home/kbunkow/cms_data/OMTF_data_2020/18_12_2020/"
trainFileNames = glob.glob(trainDataDir+'OMTFHits_pats0x0003_oldSample_files_*_chunk_0.tfrecord.gzip')

dataset = io.get_LUT_NN_dataset(batchSize, nEpochs, trainFileNames, 
                                nRefLayers=nRefLayers,
                                layer1_lut_size=layer1_lut_size,
                                layer2_lut_size=layer2_lut_size,
                                layer2_lutRangesCnt=layer2_lutRangesCnt,
                                last_input_is_bias=last_input_is_bias,
                                rangeFactor=rangeFactor,
                                isTrain=True)

Reading data from files:
/home/kbunkow/cms_data/OMTF_data_2020/18_12_2020/OMTFHits_pats0x0003_oldSample_files_30_40_chunk_0.tfrecord.gzip
/home/kbunkow/cms_data/OMTF_data_2020/18_12_2020/OMTFHits_pats0x0003_oldSample_files_15_25_chunk_0.tfrecord.gzip
/home/kbunkow/cms_data/OMTF_data_2020/18_12_2020/OMTFHits_pats0x0003_oldSample_files_1_10_chunk_0.tfrecord.gzip


2023-04-06 21:54:56.990001: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to cuInit: UNKNOWN ERROR (34)
2023-04-06 21:54:56.990054: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (73e1d29a6fac): /proc/driver/nvidia/version does not exist
2023-04-06 21:54:56.990388: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


### Model definition

In [4]:
import model_functions as models
importlib.reload(models)

model = models.get_LUT_NN(last_input_is_bias=last_input_is_bias, loss_fn=loss_fn)
model.summary()

constructing LutInterLayer  layer1 lut_size 1024 num_inputs 19 num_outputs 16 input_offset 0 self.input_offset 0 last_input_is_bias True
write_lut_hist False hist_writer None
constructing LutInterLayer  layer2 lut_size 256 num_inputs 16 num_outputs 8 input_offset 8.0 self.input_offset 8.0 last_input_is_bias False
write_lut_hist False hist_writer None
constructing LutInterLayer  layer3 lut_size 32 num_inputs 8 num_outputs 1 input_offset None self.input_offset 15.5 last_input_is_bias False
write_lut_hist False hist_writer None
layer1 
LutInterLayer.build: luts_float: layer1.luts_float:0 shape (18, 1024, 16)
Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089
layer2 
LutInterLayer.build: luts_float: layer2.luts_float:0 shape (16, 256, 8)
layer3 
LutInterLayer.build: luts_float: layer3.luts_float:0 shape (8, 32, 1)
Model: "sequential"
________

### The training loop

In [5]:
%%time
current_time = datetime.now().strftime("%Y_%b_%d_%H_%M_%S")
print("Training start. Current Time =", current_time)

nEpochs = 10

log_dir = "logs/fit/" + current_time + dir_postfix
job_dir = "training/" + current_time + dir_postfix

checkpoint_path = job_dir + "/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1,
                                                 save_freq = 5085)
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1, profile_batch=(10, 20))

model.save_weights(checkpoint_path.format(epoch=0))
   
model.fit(dataset, epochs=nEpochs, shuffle=True,
            callbacks=[tensorboard_callback, cp_callback]
            )
model.save(job_dir, save_format='tf')

current_time = datetime.now().strftime("%Y_%b_%d_%H_%M_%S")
print("Training end. Current Time =", current_time)

Training start. Current Time = 2023_Apr_06_21_54_58
Epoch 1/10


2023-04-06 21:54:58.475994: I tensorflow/core/profiler/lib/profiler_session.cc:101] Profiler session initializing.
2023-04-06 21:54:58.476032: I tensorflow/core/profiler/lib/profiler_session.cc:116] Profiler session started.
2023-04-06 21:54:58.476808: I tensorflow/core/profiler/lib/profiler_session.cc:128] Profiler session tear down.


InvalidArgumentError: Graph execution error:

indices[12] = 31 is not in [0, 31)
	 [[{{node boolean_mask/GatherV2}}]]
	 [[IteratorGetNext]] [Op:__inference_train_function_2405]